In [13]:
import tensorflow as tf

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [18]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv_1 = 32
n_conv_2 = 64

k_conv_1 = 5
k_conv_2 = 5

stride_conv_1 = 1
stride_conv_2 = 1

k_max_pool_1 = 2
k_max_pool_2 = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(k_max_pool_1*k_max_pool_2)) * (input_height//(k_max_pool_1*k_max_pool_2)) * n_conv_2

In [40]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([k_conv_1, k_conv_1, input_channels, n_conv_1])),
    "wc2" : tf.Variable(tf.random_normal([k_conv_1, k_conv_1, n_conv_1, n_conv_2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden, n_out]))    
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv_1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv_2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out]))
}

In [41]:
def cnn(x, weights, biases, keep_prob = 1.0) :
    x = tf.reshape(x, shape = [-1, input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv_1)
    conv1_pool = maxpooling(conv1, k_max_pool_1)
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv_2)
    conv2_pool = maxpooling(conv2, k_max_pool_2)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob= keep_prob)
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [42]:
"""
steps:
apply filter
add biases
apply activation function
"""
def conv(x, weights, biases, strides = 1) :
    out = tf.nn.conv2d(x, 
                       weights, 
                       padding="SAME", 
                       strides=[1, strides, strides, 1])
    out = tf.nn.bias_add(value = out, bias = biases)
    out = tf.nn.relu(out)
    return out
def maxpooling(x, k = 2) :
    return tf.nn.max_pool(x, 
                          ksize= [1, k, k, 1], 
                          padding="SAME", 
                          strides=[1, k, k, 1])

In [43]:
x = tf.placeholder(tf.float32, [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder(tf.float32)
pred = cnn(x, weights, biases, keep_prob)

In [44]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))

In [45]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [46]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [47]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches) :
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimize], feed_dict = {x:batch_x, y:batch_y, keep_prob:1.0})
        total_cost += c
    print(total_cost)

KeyboardInterrupt: 

In [ ]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions, correct_preds = sess.run([predictions, correct_predictions], feed_dict = {x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})

In [ ]:
correct_predictions.sum()/len(correct_predictions)